In [1]:
import pandas as pd
import numpy as np

import requests

import json

from datetime import datetime, timedelta

from pprint import  pprint



In [6]:
lang = input('¿En español (es) o en ingles (en)?')

restaDia = float(input('¿Cuantos días atrás?:____________'))
input_año = int(input('¿Qué año?:_____________'))

In [7]:
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Host": "apidatos.ree.es"
}

In [9]:
def generacion_datos(lang):

    now = datetime.now()
    ultima_fecha = (now - timedelta(days = restaDia)).replace(year = input_año).strftime('%Y-%m-%d')
    
    hoy = now.replace(year = input_año).strftime('%Y-%m-%d')
    
    query = f'start_date={ultima_fecha}T00:00&end_date={hoy}T23:59&time_trunc=day'


    endpoint = f"https://apidatos.ree.es/{lang}/datos/generacion/estructura-generacion?{query}"
    response = requests.get(url= endpoint, headers= headers)
    data = response.json()

    lista_nombres = list()
    lista_tipos = list()
    lista_valores = list()
    lista_porcentajes = list()
    lista_fechas = list()
    



    for dato in data['included']:
        nombre = dato['attributes']['title']
        tipo = dato['attributes']['type']


        for i in dato['attributes']['values']:
            valor = i['value']

            porcentaje = i['percentage']

            fecha = i['datetime']
            fecha = pd.to_datetime(fecha)
            

            fecha =  fecha.strftime("%d/%m/%Y")

            lista_nombres.append(nombre)
            lista_tipos.append(tipo)
            lista_valores.append(valor)
            lista_porcentajes.append(porcentaje)
            lista_fechas.append(fecha)
            

    df_generacion = pd.DataFrame()
    df_generacion['nombre']               = lista_nombres
    df_generacion['tipo de energía']      = lista_tipos
    df_generacion['Valores']              = lista_valores
    df_generacion["Porcentaje"]           = lista_porcentajes
    df_generacion["Fecha actualización"]  = lista_fechas
   


    return df_generacion
    
df_generacion = generacion_datos(lang)

df_generacion

,nombre,tipo de energía,Valores,Porcentaje,Fecha actualización
0,Hidráulica,Renovable,38951.744,0.053315,08/09/2023
1,Hidráulica,Renovable,26380.041,0.040238,09/09/2023
2,Hidráulica,Renovable,22677.543,0.035054,10/09/2023
3,Hidráulica,Renovable,34410.083,0.049715,11/09/2023
4,Hidráulica,Renovable,36184.276,0.049279,12/09/2023
...,...,...,...,...,...
523,Generación total,Generación total,720538.805,1.000000,04/10/2023
524,Generación total,Generación total,708544.054,1.000000,05/10/2023
525,Generación total,Generación total,693036.125,1.000000,06/10/2023
526,Generación total,Generación total,613026.822,1.000000,07/10/2023


In [5]:
df_generacion.to_csv('generacion_estructura.csv')